In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

# Q1

In [2]:
df_full = pd.read_parquet("../stocks_df_combined_2024_05_07.parquet.brotli")

In [3]:
df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))

/tmp/ipykernel_74590/912909511.py:1: RuntimeWarning: divide by zero encountered in log
  df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))


In [4]:
df = df_full[df_full.Date>='2000-01-01']

In [5]:
CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom']
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']

TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV
TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']
TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']
TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO
OTHER = [k for k in df_full.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]

In [6]:
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

In [7]:
df.loc[:, "month_wom"] = df.apply(
    lambda x:'%s_%d' % (x['Month'],(x['Date'].day - 1) // 7 + 1),axis=1)

/tmp/ipykernel_74590/2427115233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "month_wom"] = df.apply(


In [8]:
dummy_month_wom = pd.get_dummies(df["month_wom"], dtype='int32')

In [9]:
len(dummy_month_wom.keys().to_list())

60

In [10]:
DUMMIES = dummy_month_wom.keys().to_list()
df_with_dummies = pd.concat([df["is_positive_growth_5d_future"], dummy_month_wom], axis=1)

In [11]:
corr_is_positive_growth_5d_future = df_with_dummies.corr()['is_positive_growth_5d_future']

In [12]:
corr_is_positive_growth_5d_future_df = pd.DataFrame(corr_is_positive_growth_5d_future)

In [13]:
f'''{corr_is_positive_growth_5d_future_df[
    corr_is_positive_growth_5d_future_df.index != "is_positive_growth_5d_future"
]["is_positive_growth_5d_future"].abs().max():.3f}'''

'0.035'

# Q2

In [14]:
def temporal_split(df, min_date, max_date, train_prop=0.7, val_prop=0.15, test_prop=0.15):
    """
    Splits a DataFrame into three buckets based on the temporal order of the 'Date' column.

    Args:
        df (DataFrame): The DataFrame to split.
        min_date (str or Timestamp): Minimum date in the DataFrame.
        max_date (str or Timestamp): Maximum date in the DataFrame.
        train_prop (float): Proportion of data for training set (default: 0.6).
        val_prop (float): Proportion of data for validation set (default: 0.2).
        test_prop (float): Proportion of data for test set (default: 0.2).

    Returns:
        DataFrame: The input DataFrame with a new column 'split' indicating the split for each row.
    """
    # Define the date intervals
    train_end = min_date + pd.Timedelta(days=(max_date - min_date).days * train_prop)
    val_end = train_end + pd.Timedelta(days=(max_date - min_date).days * val_prop)

    # Assign split labels based on date ranges
    split_labels = []
    for date in df['Date']:
        if date <= train_end:
            split_labels.append('train')
        elif date <= val_end:
            split_labels.append('validation')
        else:
            split_labels.append('test')

    # Add 'split' column to the DataFrame
    df['split'] = split_labels

    return df

In [15]:
dummy_df = pd.get_dummies(df[CATEGORICAL], dtype='int32')

In [16]:
df_with_dummies = pd.concat([df, dummy_df], axis=1)

In [17]:
DUMMIES = dummy_df.keys().to_list()

In [18]:
df_with_dummies[NUMERICAL+DUMMIES].info()

<class 'pandas.core.frame.DataFrame'>
Index: 182675 entries, 3490 to 5426
Columns: 299 entries, growth_1d to month_wom_September_5
dtypes: float64(121), int32(177), int64(1)
memory usage: 294.8 MB


In [19]:
min_date_df = df_with_dummies.Date.min()
max_date_df = df_with_dummies.Date.max()

df_with_dummies = temporal_split(df_with_dummies,
                                 min_date = min_date_df,
                                 max_date = max_date_df)

In [20]:
new_df = df_with_dummies.copy()

In [21]:
new_df['pred0_manual_cci'] = (new_df.cci>200).astype(int)
new_df['pred1_manual_prev_g1'] = (new_df.growth_1d>1).astype(int)
new_df['pred2_manual_prev_g1_and_snp'] = ((new_df['growth_1d'] > 1) & (new_df['growth_snp500_1d'] > 1)).astype(int)
new_df["pred3_manual_gdp_fastd"] = ((new_df["gdppot_us_yoy"] <= 0.027) & (new_df["fastd"] >= 0.251)).astype(int)
new_df["pred4_manual_gdp_wti_oil"] = ((new_df["gdppot_us_yoy"] >= 0.027) & (new_df["growth_wti_oil_30d"] <= 1.005)).astype(int)

In [22]:
PREDICTIONS = [k for k in new_df.keys() if k.startswith('pred')]
PREDICTIONS

['pred0_manual_cci',
 'pred1_manual_prev_g1',
 'pred2_manual_prev_g1_and_snp',
 'pred3_manual_gdp_fastd',
 'pred4_manual_gdp_wti_oil']

In [23]:
# generate columns is_correct_
for pred in PREDICTIONS:
  part1 = pred.split('_')[0] # first prefix before '_'
  new_df[f'is_correct_{part1}'] =  (new_df[pred] == new_df.is_positive_growth_5d_future).astype(int)

In [24]:
IS_CORRECT =  [k for k in new_df.keys() if k.startswith('is_correct_')]
IS_CORRECT

['is_correct_pred0',
 'is_correct_pred1',
 'is_correct_pred2',
 'is_correct_pred3',
 'is_correct_pred4']

In [25]:
# define "Precision" for ALL predictions on a Test dataset (~4 last years of trading)
for i,column in enumerate(IS_CORRECT):
  prediction_column = PREDICTIONS[i]
  is_correct_column = column
  test_filter = (new_df.split=='test') & (new_df[prediction_column]==1)
  print(f'Prediction column:{prediction_column} , is_correct_column: {is_correct_column}')
  try:
      print(f"{sum(new_df[test_filter][is_correct_column] == 1)/len(new_df[test_filter]):.3}")
  except ZeroDivisionError:
      print("0.0")
      
  print('---------')

Prediction column:pred0_manual_cci , is_correct_column: is_correct_pred0
0.569
---------
Prediction column:pred1_manual_prev_g1 , is_correct_column: is_correct_pred1
0.553
---------
Prediction column:pred2_manual_prev_g1_and_snp , is_correct_column: is_correct_pred2
0.548
---------
Prediction column:pred3_manual_gdp_fastd , is_correct_column: is_correct_pred3
0.555
---------
Prediction column:pred4_manual_gdp_wti_oil , is_correct_column: is_correct_pred4
0.0
---------


# Q3

In [26]:
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [27]:
max_depth = 10
random_state = 42

In [28]:
import numpy as np

def remove_infinite_values(X):
    """
    Remove infinite values from the input array.

    Parameters:
    - X: Input array (NumPy array or array-like)

    Returns:
    - Array with infinite values removed
    """
    return X[np.isfinite(X).all(axis=1)]


In [29]:
# Split the data into training and testing sets based on the split date
features_list = NUMERICAL+DUMMIES
to_predict = 'is_positive_growth_5d_future'

train_df = new_df[new_df.split.isin(['train','validation'])].copy(deep=True)
test_df = new_df[new_df.split.isin(['test'])].copy(deep=True)

# ONLY numerical Separate features and target variable for training and testing sets
# need Date and Ticker later when merging predictions to the dataset
X_train = train_df[features_list+[to_predict,'Date','Ticker']]
X_val = train_df[train_df.split == "validation"][features_list+[to_predict,'Date','Ticker']]
X_test = test_df[features_list+[to_predict,'Date','Ticker']]

print(f'length: X_train {X_train.shape},  X_test {X_test.shape}')


length: X_train (152846, 302),  X_test (29829, 302)


In [30]:
# Can't have +-inf values . E.g. ln(volume)=-inf when volume==0 => substitute with 0

# Disable SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # default='warn'

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_val.replace([np.inf, -np.inf], np.nan, inplace=True)

# Need to fill NaNs somehow
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
X_val.fillna(0, inplace=True)

print(f'length: X_train_imputed {X_train.shape},  X_test_imputed {X_test.shape}')

length: X_train_imputed (152846, 302),  X_test_imputed (29829, 302)


In [31]:
X_train_imputed = X_train 
X_test_imputed = X_test 
X_val_imputed = X_val

In [32]:
y_train = X_train_imputed[to_predict]
y_test = X_test_imputed[to_predict]
y_val = X_val_imputed[to_predict]

# remove y_train, y_test from X_ dataframes
del X_train_imputed[to_predict]
del X_test_imputed[to_predict]
del X_val_imputed[to_predict]

In [33]:
def fit_decision_tree(X, y, max_depth=20):
# Initialize the Decision Tree Classifier
  clf = DecisionTreeClassifier(max_depth=max_depth, random_state=random_state)

  # Fit the classifier to the training data
  clf.fit(X, y)
  return clf, X.columns

In [34]:
clf_10, train_columns = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1),
                           y=y_train,
                           max_depth=10)

In [35]:
y_test_pred = clf_10.predict(X_test_imputed.drop(['Date','Ticker'],axis=1))

In [36]:
sum(
    y_test_pred * \
    test_df["is_positive_growth_5d_future"] * \
    (1 - test_df["is_correct_pred0"]) * \
    (1 - test_df["is_correct_pred1"]) * \
    (1 - test_df["is_correct_pred2"]) * \
    (1 - test_df["is_correct_pred3"]) * \
    (1 - test_df["is_correct_pred4"])
)

1

# Q4

In [39]:
try:
    X_train_imputed = X_train_imputed.drop(['Date','Ticker'],axis=1)
except:
    pass

try:
    X_val_imputed = X_val_imputed.drop(['Date','Ticker'],axis=1)
except: 
    pass

try:
    X_test_imputed = X_test_imputed.drop(['Date','Ticker'], axis=1)
except:
    pass

In [40]:
for i in range(1, 21):
    clf, train_columns = fit_decision_tree(X=X_train_imputed,
                               y=y_train,
                               max_depth=i)

    print(f"Max Depth: {i}")
    print(f"Validation Precision {precision_score(y_val, clf.predict(X_val_imputed)):.3f}")
    print(f"Test Precision {precision_score(y_test, clf.predict(X_test_imputed)):.3f}")
    print("====================")

Max Depth: 1
Validation Precision 0.572
Test Precision 0.555
Max Depth: 2
Validation Precision 0.572
Test Precision 0.555
Max Depth: 3
Validation Precision 0.571
Test Precision 0.555
Max Depth: 4
Validation Precision 0.571
Test Precision 0.555
Max Depth: 5
Validation Precision 0.571
Test Precision 0.555
Max Depth: 6
Validation Precision 0.570
Test Precision 0.571
Max Depth: 7
Validation Precision 0.572
Test Precision 0.568
Max Depth: 8
Validation Precision 0.576
Test Precision 0.568
Max Depth: 9
Validation Precision 0.577
Test Precision 0.570
Max Depth: 10
Validation Precision 0.585
Test Precision 0.565
Max Depth: 11
Validation Precision 0.602
Test Precision 0.567
Max Depth: 12
Validation Precision 0.615
Test Precision 0.572
Max Depth: 13
Validation Precision 0.620
Test Precision 0.571
Max Depth: 14
Validation Precision 0.643
Test Precision 0.584
Max Depth: 15
Validation Precision 0.666
Test Precision 0.586
Max Depth: 16
Validation Precision 0.673
Test Precision 0.572
Max Depth: 17
Val